In [1]:
pip install scikit-surprise

  Using cached scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('linkdin_job_posting.csv')

# Display the first few rows to understand the structu
df.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,min_salary,formatted_work_type,applies,original_listed_time,remote_allowed,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,17.0,Full-time,2.0,1.713400e+12,NaN,https://www.linkedin.com/jobs/view/921716/?trk...,NaN,ComplexOnsiteApply,1.715990e+12,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,1.713400e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,30.0,Full-time,NaN,1.712860e+12,NaN,https://www.linkedin.com/jobs/view/1829192/?tr...,NaN,ComplexOnsiteApply,1.715450e+12,NaN,NaN,NaN,1.712860e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,45000.0,Full-time,NaN,1.713280e+12,NaN,https://www.linkedin.com/jobs/view/10998357/?t...,NaN,ComplexOnsiteApply,1.715870e+12,NaN,NaN,We are currently accepting resumes for FOH - A...,1.713280e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,140000.0,Full-time,NaN,1.712900e+12,NaN,https://www.linkedin.com/jobs/view/23221523/?t...,NaN,ComplexOnsiteApply,1.715490e+12,NaN,NaN,This position requires a baseline understandin...,1.712900e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,60000.0,Full-time,NaN,1.713450e+12,NaN,https://www.linkedin.com/jobs/view/35982263/?t...,NaN,ComplexOnsiteApply,1.716040e+12,NaN,NaN,NaN,1.713450e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [5]:
# Selecting relevant columns for the recommendation task
relevant_columns = ['title', 'views', 'applies']

# Dropping rows with missing values in the relevant columns
cleaned_data = df[relevant_columns].dropna()

# Display the cleaned dataset to confirm
cleaned_data.head()

,title,views,applies
0,Marketing Coordinator,20.0,2.0
5,Economic Development and Planning Intern,9.0,4.0
6,Producer,7.0,1.0
10,Inside Customer Service Associate,4.0,1.0
18,"Content Writer, Communications",10.0,1.0


In [6]:
cleaned_data.shape

(23319, 3)

In [7]:
# For simplicity, we'll simulate 1000 unique users
np.random.seed(42)  # For reproducibility
num_users = 1000
user_ids = np.random.randint(1, num_users + 1, size=len(cleaned_data))

In [8]:
# Add the simulated User_ID to the dataset
cleaned_data['User_ID'] = user_ids

In [9]:
#Fill missing values in 'applies' or 'views' with 0 (representing no interaction)
cleaned_data['applies'].fillna(0, inplace=True)
cleaned_data['views'].fillna(0, inplace=True)

In [10]:
#Decide which interaction to use (we'll use 'applies' for this example)
cleaned_data['interaction'] = cleaned_data['applies']

In [11]:
# Drop rows where interaction data is missing or 0 (optional step based on model needs)
cleaned_data = cleaned_data[cleaned_data['interaction'] > 0]
cleaned_data.head()

,title,views,applies,User_ID,interaction
0,Marketing Coordinator,20.0,2.0,103,2.0
5,Economic Development and Planning Intern,9.0,4.0,436,4.0
6,Producer,7.0,1.0,861,1.0
10,Inside Customer Service Associate,4.0,1.0,271,1.0
18,"Content Writer, Communications",10.0,1.0,107,1.0


In [12]:
# Final dataset ready for SVD training
cleaned_data = cleaned_data[['User_ID', 'title', 'interaction']]

cleaned_data.head()

,User_ID,title,interaction
0,103,Marketing Coordinator,2.0
5,436,Economic Development and Planning Intern,4.0
6,861,Producer,1.0
10,271,Inside Customer Service Associate,1.0
18,107,"Content Writer, Communications",1.0


In [13]:
# Save the cleaned dataset (optional, if you want to save the preprocessed data)
cleaned_data.to_csv('cleaned_job_posting_for_svd_use_title.csv', index=False)

**Model Training**

In [14]:
# Check the range of 'applies' to determine the correct rating scale
print(cleaned_data['interaction'].min(), cleaned_data['interaction'].max())


1.0 967.0


In [15]:
# Step 1: Prepare the data for SVD in Surprise format
reader = Reader(rating_scale=(1, 967))  # Assuming the rating scale ranges from 1 to 10
data = Dataset.load_from_df(cleaned_data[['User_ID', 'title', 'interaction']], reader)

In [16]:
# Step 2: Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

In [17]:
# Step 3: Train the SVD model
model = SVD()
model.fit(trainset)

In [18]:
# Step 4: Evaluate the model on the test set using RMSE (optional)
predictions = model.test(testset)
accuracy.rmse(predictions)

RMSE: 956.8663


956.8663014518415

In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Extract the true ratings and predicted ratings
true_ratings = [pred.r_ui for pred in predictions]  # Actual ratings
predicted_ratings = [pred.est for pred in predictions]  # Predicted ratings

# Mean Absolute Error (MAE)
mae = mean_absolute_error(true_ratings, predicted_ratings)

# Mean Squared Error (MSE)
mse = mean_squared_error(true_ratings, predicted_ratings)

# R-squared (R²)
r2 = r2_score(true_ratings, predicted_ratings)

# RMSE
rmse = accuracy.rmse(predictions)

# Print the evaluation metrics
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")

RMSE: 956.8663
RMSE: 956.8663014518415
MAE: 956.000264025602
MSE: 915593.1188541264
R²: -1338.1434774053548


I will run hyperparameter optimization

In [20]:
pip install optuna

  Using cached optuna-4.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached alembic-1.13.3-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached Mako-1.3.5-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-4.0.0-py3-none-any.whl (362 kB)
Using cached alembic-1.13.3-py3-none-any.whl (233 kB)
Using cached colorlog-6.8.2-py3-none-any.whl (11 kB)
Using cached Mako-1.3.5-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [21]:
import optuna
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
import pandas as pd

# Prepare the data for Surprise
reader = Reader(rating_scale=(1, 967))  # Assuming the rating scale ranges from 1 to 10
data = Dataset.load_from_df(cleaned_data[['User_ID', 'title', 'interaction']], reader)

# Split the data into trainset and testset (this will be done within the cross-validation)
trainset = data.build_full_trainset()

# Objective function for Optuna
def objective(trial):
    # Define hyperparameters to be tuned
    param = {
        'n_factors': trial.suggest_int('n_factors', 10, 200),  # Number of latent factors
        'n_epochs': trial.suggest_int('n_epochs', 5, 50),  # Number of epochs
        'lr_all': trial.suggest_float('lr_all', 0.001, 0.05),  # Learning rate for SGD
        'reg_all': trial.suggest_float('reg_all', 0.001, 0.1)  # Regularization term
    }

    # Create the SVD model with the suggested hyperparameters
    svd_model = SVD(**param)

    # Perform cross-validation and return the RMSE score
    results = cross_validate(svd_model, data, measures=['rmse'], cv=5, verbose=False)
    
    # Return the average RMSE score (minimizing this)
    return np.mean(results['test_rmse'])

# Enable the default logger of Optuna
optuna.logging.enable_default_handler()

# Set the logging level (for more detailed information)
optuna.logging.set_verbosity(optuna.logging.INFO)

# Create an Optuna study for minimizing the RMSE (Root Mean Squared Error)
study = optuna.create_study(direction='minimize')

# Perform optimization
study.optimize(objective, n_trials=100)

# Best hyperparameters
print('Best trial:', study.best_trial.params)

# Best RMSE value
print('Best RMSE:', study.best_value)



[I 2024-10-20 08:12:16,912] A new study created in memory with name: no-name-d0fcb084-5a64-4a19-914a-a9bd681ac694
[I 2024-10-20 08:12:22,574] Trial 0 finished with value: 956.8277692552526 and parameters: {'n_factors': 189, 'n_epochs': 40, 'lr_all': 0.017126665373013876, 'reg_all': 0.053699161003071566}. Best is trial 0 with value: 956.8277692552526.
[I 2024-10-20 08:12:26,412] Trial 1 finished with value: 956.6644504512208 and parameters: {'n_factors': 120, 'n_epochs': 34, 'lr_all': 0.02559372105183414, 'reg_all': 0.02581447008128872}. Best is trial 1 with value: 956.6644504512208.
[I 2024-10-20 08:12:30,166] Trial 2 finished with value: 956.6406811692947 and parameters: {'n_factors': 138, 'n_epochs': 30, 'lr_all': 0.015349887120419965, 'reg_all': 0.09754759255065265}. Best is trial 2 with value: 956.6406811692947.
[I 2024-10-20 08:12:32,105] Trial 3 finished with value: 956.7655550095333 and parameters: {'n_factors': 144, 'n_epochs': 11, 'lr_all': 0.017158725798386853, 'reg_all': 0.0

Best trial: {'n_factors': 130, 'n_epochs': 6, 'lr_all': 0.0011058014999946024, 'reg_all': 0.026351315219637125}
Best RMSE: 28.68198496478479


In [22]:
best_params = study.best_trial.params
print("Best hyperparameters: ", best_params)

Best hyperparameters:  {'n_factors': 130, 'n_epochs': 6, 'lr_all': 0.0011058014999946024, 'reg_all': 0.026351315219637125}


In [23]:
# retrain the model with the tuned hyperparameters

# Step 3: Train the SVD model
model_tune = SVD(**best_params)
model_tune.fit(trainset)

In [24]:
# Step 4: Evaluate the model on the test set using RMSE (optional)
predictions = model_tune.test(testset)
accuracy.rmse(predictions)

RMSE: 21.8510


21.850985919614807

In [25]:
# Extract the true ratings and predicted ratings
true_ratings = [pred.r_ui for pred in predictions]  # Actual ratings
predicted_ratings = [pred.est for pred in predictions]  # Predicted ratings

# Mean Absolute Error (MAE)
mae = mean_absolute_error(true_ratings, predicted_ratings)

# Mean Squared Error (MSE)
mse = mean_squared_error(true_ratings, predicted_ratings)

# R-squared (R²)
r2 = r2_score(true_ratings, predicted_ratings)

# RMSE
rmse = accuracy.rmse(predictions)

# Print the evaluation metrics
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")

RMSE: 21.8510
RMSE: 21.850985919614807
MAE: 10.511682324075766
MSE: 477.4655856592046
R²: 0.3016604083686645


The hyperparameter tuning improved the model

In [26]:
# Step 5: Save the trained model using joblib for later use (e.g., deployment)

import joblib
model_filename = 'svd_model_job_use_title.joblib'
joblib.dump(model_tune, model_filename)
print(f"Model saved as {model_filename}")

Model saved as svd_model_job_use_title.joblib
